<a href="https://colab.research.google.com/github/Nimesh33390/Fine-tuning/blob/main/fine_tuning_distilbert_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install transformers datasets evaluate

In [10]:
!pip install --upgrade datasets transformers huggingface_hub

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
# from datasets import load_dataset
# dataset = load_dataset("imdb")
from datasets import load_dataset
dataset = load_dataset("imdb", split={"train": "train", "test": "test"})

In [3]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [4]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(20000)),
    eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(5000)),
    compute_metrics=compute_metrics,
)

In [15]:
import wandb # Import wandb

# Initialize wandb
wandb.init(project="imdb-sentiment-analysis") # Replace "imdb-sentiment-analysis" with your desired project name

trainer.train()

# Optional: End the wandb run when training is finished
wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nimesh2767 (nimesh2767-sardar-vallabhbhai-national-institute-of-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.248700,0.197276,0.924000
2,0.123700,0.262153,0.925200
3,0.058100,0.344077,0.925400


eval/accuracy,▁▇█
eval/loss,▁▄█
eval/runtime,█▄▁
eval/samples_per_second,▁▅█
eval/steps_per_second,▁▅█
train/epoch,▁▂▃▃▄▅▅▆▇██
train/global_step,▁▂▃▃▄▅▅▆▇██
train/grad_norm,▆█▂▁▁▃▁
train/learning_rate,█▇▆▅▃▂▁
train/loss,▃█▅▃▃▁▁
eval/accuracy,0.9254


In [17]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

# Assuming 'trainer' is your Trainer instance
trainer.save_model("/content/model")  # Saves the model

# Save the tokenizer
trainer.tokenizer.save_pretrained("/content/model")


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


AttributeError: 'NoneType' object has no attribute 'save_pretrained'